In [166]:
import json
import pandas as pd
from ap_rllib.config import get_config_by_name
from os.path import expanduser
from ap_rllib_experiment_analysis.analysis_helpers import get_df_from_logdir, read_json_array_flat
import os

In [175]:
# prune empty checkpoints
# for a in train*;do n=$(cat $a/result.json|wc -l); if (( $n < 10 )); then rm -rf $a;fi;echo $a $n;done

In [ ]:
# Don't forget to ln new results dir to old results dir

In [183]:
def f_count_lines(fn):
    """Count lines in a given file."""
    counter = 0
    with open(fn, 'r') as f:
        for line in f.readlines():
            counter += 1
    return counter

In [192]:
def overwrite_state(trial_name):
    """Convert ray 0.8.6 checkpoint to ray 1.0.1 checkpoint."""
    home = expanduser("~")
    path = os.path.join(home, 'ray_results', trial_name)
    exp_states = sorted([x for x in os.listdir(path) if x.startswith("experiment_state")])
    state_file = exp_states[0]
    state_file = os.path.join(path, state_file)
    with open(state_file, 'r') as f:
        state = json.load(f)
        
    checkpoints_export = []
    for ckpt in state['checkpoints']:
        if ckpt['status'] == 'ERROR':
#             print("Overwriting ERROR to RUNNING")
            ckpt['status'] = 'RUNNING'
#         print(ckpt['status'])
        i = None
        if 'training_iteration' in ckpt['last_result']:
            i = ckpt['last_result']['trainer_iteration']
#         print(ckpt)

        logdir = ckpt['logdir']
        lines = 0
        if logdir:
            lines = f_count_lines(logdir + '/result.json')

        print(i, ckpt['trial_id'], ckpt['status'], lines)
            
        
        ckpt['keep_checkpoints_num'] = None
        ckpt['checkpoint_score_attr'] = "training_iteration"
        ckpt["custom_dirname"] = None
        ckpt["trial_name_creator"] = None
        ckpt["log_to_file"] = [None, None]
        if "checkpoint_manager" in ckpt:
            del ckpt["checkpoint_manager"]
        if lines > 10:
            checkpoints_export.append(ckpt)

    state['checkpoints'] = checkpoints_export
        
    with open(state_file, 'w') as f:
        json.dump(state, f)

In [193]:
overwrite_state("bursts_exp_withnormal_pbt_sb")

5852 ad2b1_00000 RUNNING 5852
5840 ad2b1_00001 RUNNING 5840
None ad2b1_00002 PENDING 6553
None ad2b1_00003 PENDING 6558
None ad2b1_00004 PENDING 6552
None ad2b1_00005 PENDING 6547
5821 ad2b1_00006 RUNNING 5821
None ad2b1_00007 PENDING 6558
None ad2b1_00008 PENDING 6547
None ad2b1_00009 PENDING 6566
None ad2b1_00010 PENDING 709
None ad2b1_00011 PENDING 709
None ad2b1_00012 PENDING 710


In [30]:
def get_first_state_from_name(trial_name):
    home = expanduser("~")
    path = os.path.join(home, 'ray_results', trial_name)
    exp_states = sorted([x for x in os.listdir(path) if x.startswith("experiment_state")])
    state_file = exp_states[0]
    state = json.load(open(os.path.join(path, state_file), 'r'))
    return state

In [105]:
n1 = "tst1"
n2 = "bursts_exp_withnormal_pbt_sb"

state1 = get_first_state_from_name(n1)
state2 = get_first_state_from_name(n2)

In [106]:
def check1(state):
    return state['runner_data']
def check2(state):
    return state['checkpoints'][1]

In [107]:
for check in [check1, check2]:
    for key in set(check(state1).keys()):
        if key not in set(check(state2).keys()):
            print(key, check(state1)[key])

_metric None


In [108]:
k2 = state['runner_data'].keys()

In [63]:
for ckpt in state2'checkpoints']:
    print(ckpt['status'])
    print(ckpt['state'])

PENDING


KeyError: 'state'

In [145]:
checkpoint

{'trainable_name': 'train_one_with_sacred',
 'trial_id': 'ad2b1_00000',
 'config': {'kl_coeff': 1.0,
  '_num_workers_tf': 4,
  'use_gae': True,
  'num_gpus': 0,
  '_env_name_rllib': 'multicomp',
  '_env_fcn': {'_type': 'CLOUDPICKLE_FALLBACK',
   'value': '80059524000000000000008c1167796d5f636f6d706574655f726c6c6962948c0a6372656174655f656e769493942e'},
  '_policies': [None, 'from_scratch', 'pretrained'],
  '_env': {'with_video': False,
   'SingleAgentToMultiAgent': False,
   'env_name': 'multicomp/YouShallNotPassHumans-v0'},
  'framework': 'tfe',
  '_train_policies': [],
  '_call': {'checkpoint_freq': 0,
   'name': 'adversarial_tune_bursts_exp_withnormal_pbt_sb',
   'stop': {'timesteps_total': 204800000},
   'resources_per_trial': {'custom_resources': {'tune_cpu': 6}},
   'num_samples': 100},
  '_trainer': 'External',
  '_policy': 'PPO',
  '_train_steps': 10000,
  '_update_config': None,
  '_run_inline': False,
  '_postprocess': None,
  'num_envs_per_worker': 10,
  '_log_error': True,
 